# Exploratory Data Analysis: Online Retail Dataset  
*Author: Pragun Sapotra*  
*Date: August 2025*

This notebook explores the "Online Retail" dataset to understand data structure, types, and basic statistics.  
It sets the foundation for subsequent data cleaning and analysis tasks.


## Importing libraries

In [ ]:
# Environment Information and Setup

import sys
print(f"Python version: {sys.version}")

from IPython.display import display

import pandas as pd
print(f"Pandas version: {pd.__version__}")

import numpy as np
print(f"Numpy version: {np.__version__}")

import matplotlib
print(f"Matplotlib version: {matplotlib.__version__}")



# Uncomment and run if needed to install packages
# !pip install pandas numpy matplotlib openpyxl


## Dataset Loading

I define the file path relative to the current working directory for portability.  
Next, I retrieve and print the sheet names in the Excel file to confirm the available data.  
Finally, I load the 'Online Retail' sheet into a pandas DataFrame (`raw_df`) and make a copy (`df`) for analysis.


In [ ]:
base_dir = os.getcwd()
file_path = os.path.join(base_dir, "../Data/global_indicators_raw.xlsx")

xlsx = pd.ExcelFile(file_path)
print(xlsx.sheet_names)

raw_df = pd.read_excel(file_path, sheet_name='Online Retail')
df = raw_df.copy()

## Initial Data Exploration

### Check Dataset Shape (Rows & Columns)

In [ ]:
df.shape

### View First and Last Few Rows (head(), tail())

In [ ]:
print("First 5 rows:")
display(df.head())

print("Last 5 rows:")
display(df.tail())


In [ ]:
df.tail()

### View Column Names and Data Types

In [ ]:
df.info()

### Initial Observations:
- Columns `Description` and `CustomerID` contain missing values that will require cleaning.
- `CustomerID` is stored as a float due to the presence of missing values (NaNs), although customer IDs are typically integers or strings.
- The `InvoiceNo` column contains many repeated values, likely indicating multiple items per invoice or potential duplicates, which needs further investigation.


## Handling Duplicates
In this section, I check for duplicate rows, inspect them, and filter specific cases for further investigation.


### Identify Duplicate Rows

In [ ]:
df.duplicated().sum()

The dataset contains 5268 fully duplicated rows.


In [ ]:
df[df.duplicated()]

Viewing the duplicate rows reveals that they have identical values across all columns.

### Filter Specific Duplicate Cases for Investigation
Example: Filtering duplicates with StockCode = 22111 and InvoiceNo = 536409.


In [ ]:
df[ (df["StockCode"] == 22111) & (df["InvoiceNo"] == 536409	) ]

### Removing Duplicate Rows


In [ ]:
df.drop_duplicates(inplace=True)

All exact duplicate rows were removed across the entire dataset using `drop_duplicates()` to ensure data integrity.  
The first occurrence of each duplicated record was retained while subsequent duplicates were dropped.

- Total rows before cleaning: 541,909  
- Total rows after cleaning: 536,641


## Missing Values Analysis & Cleaning

### Count Missing Values per Column
Checking for null values in each column to identify data gaps before cleaning.


In [ ]:
df.isnull().sum()

### Missing Values Overview
- `Description`: 1,454 missing entries.
- `CustomerID`: 135,037 missing entries — likely guest or anonymous customers.
- All other columns have no missing values.

### Inspect `Description` Column

In [ ]:
df[df['Description'].isnull()]

**Observations on Missing `Description` Values:**  
- All rows retrieved with `Above command` have a **`UnitPrice` of 0.0**.  
- All of them also have a **missing (`NaN`) `CustomerID`**.  
- At least one row has a **negative `Quantity`**, which could indicate a return.  
- This suggests these records might represent **invalid or placeholder transactions**, but this needs full validation before deciding how to handle them.  


In [ ]:
df[(df["Description"].isnull()) & (df["UnitPrice"]== 0.0) & (df["CustomerID"].isnull())]

**Observations on Missing `Description` Values:**  
- All 1454 rows with missing `Description` also have:  
  - **`UnitPrice` of 0.0**  
  - **Missing (`NaN`) `CustomerID`**  
  - The same **`StockCode` = 22139**  
- These rows appear to be non-usable for analysis and may need to be removed.  

In [ ]:
df.loc[df["StockCode"] == 22139, "Description"].dropna().unique()

**Investigation of Missing `Description` Values:**
- Checked if missing `Description` rows could be filled using their `StockCode`.
- Found that `StockCode 22139` (which appears in all such rows) maps to **multiple different descriptions** in the dataset.
- Since there is no reliable 1-to-1 mapping, filling these values could introduce incorrect data.
- Decision: **Drop these rows** instead of imputing a value.


### Handle Missing Values

In [ ]:
df.drop(df[
    (df["Description"].isnull()) &
    (df["UnitPrice"] == 0.0) &
    (df["CustomerID"].isnull())
].index, inplace=True)

**Removed Invalid Placeholder Transactions**  
Dropped rows where:  
- `Description` is missing (`NaN`)  
- `UnitPrice` equals `0.0`  
- `CustomerID` is missing (`NaN`)  

These records appear to be non-usable placeholder entries.  

- **Total rows before:** 536,641  
- **Total rows after:** 535,187  
- **Rows removed:** 1,454  


### Inspect `CustomerID` Column

In [ ]:
df[df["CustomerID"].isnull()]

- Missing CustomerID (~133K rows) retained as NaN to preserve data and numeric type integrity.
- Detailed reasoning provided in project README.


## Data Type Validation

Ensure all columns have the correct data types before performing further analysis.

In [ ]:
df.info()

All columns have valid data types. No corrections required.

## Descriptive statistics

### Detect Outliers Using IQR (Interquartile Range)

In [ ]:
def detect_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    LB = Q1 - 1.5 * IQR
    UB = Q3 + 1.5 * IQR

    IQR_Outliers = series[(series < LB) | (series > UB)]
    return IQR_Outliers, LB, UB

# UnitPrice :
IQR_Outliers, LB, UB = detect_outliers_iqr(df["UnitPrice"])

print("Lower Bound:", LB)
print("Upper Bound:", UB)
print(f"Outliers detected: {len(IQR_Outliers)} rows")

df_outliers = df[df["UnitPrice"].isin(IQR_Outliers)]
df_outliers

### Z-score

In [ ]:
def detect_outliers_z(series, threshold=3):

    mean = series.mean()
    std = series.std()

    df["z_score"] = (series - mean) / std

    Z_Outliers = series[np.abs(df["z_score"]) > threshold]
    Z_Outliers_df = df.loc[Z_Outliers.index]


    return Z_Outliers, Z_Outliers_df, mean, std


# Detect outliers in UnitPrice
Z_Outliers, Z_Outliers_df, mean, std = detect_outliers_z(df["UnitPrice"], threshold=3)

print("Mean:", mean)
print("Standard Deviation:", std)
print(f"Outliers detected: {len(Z_Outliers)} rows")

Z_Outliers_df


## Graphs

### Histogram: Distribution of Unit Prices

In [ ]:
# Histogram to visualize distribution of UnitPrice

plt.hist(df["UnitPrice"] , bins = 30, edgecolor='Black')
plt.grid(True, linestyle='--', alpha=0.5)
plt.yscale('log')
plt.title('Distribution of Unit Prices')
plt.xlabel('Unit Price')
plt.ylabel('frequency')
plt.show()

### Violin Plot: Distribution of Z-Scores

In [ ]:
plt.figure(figsize=(6,3))
plt.violinplot(df["z_score"], showextrema=True)

plt.axhline(3, color="green", linestyle="--", linewidth=1, label="Threshold (+3)")
plt.axhline(-3, color="red", linestyle="--", linewidth=1, label="Threshold (-3)")
plt.legend(loc="upper right", fontsize=6.5)

plt.title("Distribution of Z-Scores")
plt.ylabel("Z-Score")

plt.show()


## Handling Extreme Outliers and Data Filtering


### Handling extreme positive `Outliers`

Inspect top 20 extreme positive outliers using Z-score

In [ ]:
df.sort_values(by="z_score", ascending=False).head(20)

### Handling extreme negative `Outliers`

Inspect top 20 extreme negative outliers using Z-score

In [ ]:
df.sort_values(by="z_score", ascending=True).head(20)

Remove known problematic invoices identified as extreme negative outliers in UnitPrice

In [ ]:
df = df[~df["InvoiceNo"].isin(['A563187', 'A563186'])]

### Handling extreme negative `Quantity`

Inspect negative quantities in Quantity column

In [ ]:
negative_qty = df[df['Quantity'] < 0]

print(negative_qty.shape)
print(negative_qty['Quantity'].min(), negative_qty['Quantity'].max())


Filter out extreme negative quantities, keeping only normal returns (Quantity >= -10)

In [ ]:
df = df[df['Quantity'] >= -10]

## Final Distribution of UnitPrice after Cleaning Extreme Outliers and Problematic Invoices

In [ ]:
plt.hist(df["UnitPrice"] , bins = 30, edgecolor='Black')
plt.grid(True, linestyle='--', alpha=0.5)
plt.yscale('log')
plt.title('Distribution of Unit Prices')
plt.xlabel('Unit Price')
plt.ylabel('Frequency')
plt.show()